In [21]:
import pandas as pd
import cv2
import numpy as np
from PIL import Image
from numba.experimental import jitclass
import os
import re

In [22]:
rgb_data = pd.read_csv('data/avg_database.csv')
rgb_data

,Filename,R Average,G Average,B Average
0,1593190.jpg,126.914235,102.113993,102.803323
1,2336660.jpg,88.919509,92.358368,94.949701
2,2364278.jpg,90.594624,77.809323,78.683701
3,2324905.jpg,163.152584,152.239820,136.983652
4,2410442.jpg,104.139373,114.299440,113.638524
...,...,...,...,...
67679,cifar_test_9995.jpeg,93.085938,123.327148,83.829102
67680,cifar_test_9996.jpeg,120.177734,132.110352,79.393555
67681,cifar_test_9997.jpeg,75.832031,106.240234,64.107422
67682,cifar_test_9998.jpeg,104.074219,95.823242,79.599609


In [23]:
def calc_l2_dist(a, b, c, x, y, z):  
    return (abs(int(a)-int(x)) + abs(b-y) + abs(c-z))

In [24]:
def remove_duplicate(data, image):
    vals = (data['Filename']==image[0]).values
    idx = np.where(vals == True)
    idx = int(idx[0])
    data = data.drop([idx])
    
    return data

In [25]:
def find_nearest_img(r, g, b, data):
    least_dist = np.inf
    least_row = None
    
    tmp_dist = 0
    tmp_r1 = 0
    tmp_r2 = 0
    tmp_r3 = 0
    
    for row in data.values:
        dist = calc_l2_dist(r, g, b, row[1], row[2], row[3])
        tmp_dist = dist
        tmp_r1 = row[1]
        tmp_r2 = row[2]
        tmp_r3 = row[3]
        if dist < least_dist:
            least_dist = dist
            least_row = row
            
            
    try:
        file = least_row[0]
    except:
        print('least row: ', least_row)
        print('least dist: ', least_dist)
        print('r: ', tmp_r1)
        print('g: ', tmp_r2)
        print('b: ', tmp_r3)
        print('df length: ', len(data))
        
        
    # remove dupes
    data = data.drop(data.index[data['Filename'] == file].tolist())
    return file, data

In [26]:
def construct_mosaic(pixel_batch, r, g, b, data):
    # create empty matrix of target image size
    mosaic = np.zeros((r.shape[0], r.shape[1], 3))
    
    max_x = r.shape[0]/pixel_batch
    max_y = r.shape[1]/pixel_batch
    print('max images along x: ', max_x)
    print('max images along y: ', max_y)
    
    # iterate through RGB and find the closest related image
    for x in range(0, r.shape[0], pixel_batch):
        for y in range(0, r.shape[1], pixel_batch):

            # need to handle padding issues later
            r_batch = r[x:x+pixel_batch, y:y+pixel_batch]
            g_batch = b[x:x+pixel_batch, y:y+pixel_batch]
            b_batch = g[x:x+pixel_batch, y:y+pixel_batch]
            
            # computer batch averages
            r_avg = np.mean(r_batch)
            g_avg = np.mean(g_batch)
            b_avg = np.mean(b_batch)
            
            # find the closest image
            # returns the filename
            img,data = find_nearest_img(r_avg, b_avg, g_avg, data)
            tmp_img_name = img
            # print(len(data))
            
            # extract the filename
            # img = img[0]
            img = Image.open('data/images/'+img)
            tmp_img = img
            
            # resize the image to be the batch size
            img = img.resize((pixel_batch, pixel_batch))#, Image.ANTIALIAS)
            
            # Convert to opencv image
            img = np.array(img) 
            try:
                img = img[:, :, ::-1].copy() 
            except:
                print(tmp_img_name)
                tmp_img.show()
            
            # add the image to the mosaic
            mosaic[x:x+pixel_batch, y:y+pixel_batch, 0] = img[:,:,2]
            mosaic[x:x+pixel_batch, y:y+pixel_batch, 1] = img[:,:,1]
            mosaic[x:x+pixel_batch, y:y+pixel_batch, 2] = img[:,:,0]
    
    out = Image.fromarray(mosaic.astype(np.uint8))
    return out

In [27]:
#read in the target image and divide it into RGB channels
def extract_rgb(target):
    image = cv2.imread(target)
    b,g,r = cv2.split(image)

    print('R: ', r.shape)
    print('G: ', g.shape)
    print('B: ', b.shape)
    
    return[r, g, b]

In [ ]:
# assign a pixel batch size
pixel_batch = 50
data = rgb_data.copy()
target = 'data/flower.jpg'

r,g,b = extract_rgb(target)
image = construct_mosaic(pixel_batch, r, g, b, data)
image.show()
image.save('data/flower_norepeats_pb50.jpg')

In [ ]:
pixel_batch = 20
data = rgb_data.copy()
target = 'data/flower.jpg'

r,g,b = extract_rgb(target)
image = construct_mosaic(pixel_batch, r, g, b, data)
image.show()
image.save('dataflower_norepeats_pb20.jpg')

R:  (1200, 1600)
G:  (1200, 1600)
B:  (1200, 1600)
max images along x:  60.0
max images along y:  80.0


In [ ]:
pixel_batch = 10
data = rgb_data.copy()
target = 'data/flower.jpg'

r,g,b = extract_rgb(target)
image = construct_mosaic(pixel_batch, r, g, b, data)
image.show()
image.save('data/flower_norepeats_pb10.jpg')

In [19]:
pixel_batch = 20
data = rgb_data.copy()

for frame in os.listdir('data/videos/bunny_frames/'):
    # print(frame)
    file = 'data/videos/bunny_frames/' + frame
    r,g,b = extract_rgb(file)
    image = construct_mosaic(pixel_batch, r, g, b, data)
    regex = re.compile(r'\d+')
    frame_num = regex.findall(frame)[0]
    image.save('data/videos/mosaic_bunny_frames/frame%d.jpg' % int(frame_num))

R:  (720, 1280)
G:  (720, 1280)
B:  (720, 1280)
max images along x:  36.0
max images along y:  64.0
R:  (720, 1280)
G:  (720, 1280)
B:  (720, 1280)
max images along x:  36.0
max images along y:  64.0
R:  (720, 1280)
G:  (720, 1280)
B:  (720, 1280)
max images along x:  36.0
max images along y:  64.0
R:  (720, 1280)
G:  (720, 1280)
B:  (720, 1280)
max images along x:  36.0
max images along y:  64.0
R:  (720, 1280)
G:  (720, 1280)
B:  (720, 1280)
max images along x:  36.0
max images along y:  64.0
R:  (720, 1280)
G:  (720, 1280)
B:  (720, 1280)
max images along x:  36.0
max images along y:  64.0
R:  (720, 1280)
G:  (720, 1280)
B:  (720, 1280)
max images along x:  36.0
max images along y:  64.0
R:  (720, 1280)
G:  (720, 1280)
B:  (720, 1280)
max images along x:  36.0
max images along y:  64.0
R:  (720, 1280)
G:  (720, 1280)
B:  (720, 1280)
max images along x:  36.0
max images along y:  64.0
R:  (720, 1280)
G:  (720, 1280)
B:  (720, 1280)
max images along x:  36.0
max images along y:  64.0


KeyboardInterrupt: 

In [ ]:
idx = data.index[data['Filename'] == '2338347.jpg'].tolist()
data.loc[idx]